In [1]:
import numpy as np
import tensorflow as tf
import glob
import PIL
from PIL import Image  
from skimage import color
import shutil
import json
import os

In [2]:
def preprocessing_image(img_path,image_size):
    original = Image.open(img_path)  
    resized_image = original.resize(image_size)
    resized_image.load()
    data = np.asarray( resized_image, dtype="int32" )
    grayscale_image = color.rgb2gray(data)  
    return grayscale_image

In [3]:
def data_generation(img_dir,resize_image):
    print(resize_image)
    image_data = []
    img_label = []
    label_counter=0
    for each_dir in sorted(glob.glob(img_dir+"\\"+"*"+"\\")):
        for each in sorted(glob.glob(each_dir+"*.PNG")):
            preprocessed_image = preprocessing_image(each,resize_image)
            image_data.append(preprocessed_image)
            img_label.append(int(os.path.basename(os.path.dirname(each))))
    return np.array(image_data), np.array(img_label)

In [4]:
def train_test_divider(dir_label, img_dir, train_obj_dir, test_obj_dir, train_dir, test_dir):
    train_data = [os.path.basename(i).split(".")[0] for i in glob.glob(train_obj_dir+"*.JPG")]
    test_data = [os.path.basename(i).split(".")[0] for i in glob.glob(test_obj_dir+"*.JPG")]
    for each in glob.glob(img_dir+"*.PNG"):
        file_basename = os.path.basename(each).split(".")[0]
        if file_basename in train_data:
            try:
                os.mkdir(train_dir + "\\" + dir_label) 
            except FileExistsError:
                pass
            shutil.copy2(each,train_dir + "\\" + dir_label)
        elif file_basename in test_data:
            try:
                os.mkdir(test_dir + "\\" + dir_label) 
            except FileExistsError:
                pass
            shutil.copy2(each,test_dir + "\\" + dir_label)
    print("Train Test Split Complete")

In [5]:
def data_preparation_pipeline():
    with open('config.json') as json_file:
        data = json.load(json_file)
#     all_label_dir = glob.glob(data["object_classifier_directory"]["img_dir"]+"*"+"\\")
#     try:
#         os.mkdir(data["object_classifier_directory"]["train_img_dir"])
#         os.mkdir(data["object_classifier_directory"]["test_img_dir"])
#     except FileExistsError:
#         pass
#     for each in all_label_dir:
#         train_test_divider(os.path.basename(os.path.dirname(each)), 
#                            each,
#                            data["object_detector_directory"]["train_object_dir"],
#                            data["object_detector_directory"]["test_object_dir"],
#                            data["object_classifier_directory"]["train_img_dir"],
#                            data["object_classifier_directory"]["test_img_dir"],
#                            )
    train_data, train_label = data_generation(data["object_classifier_directory"]["train_img_dir"],(55,27))
    test_data, test_label = data_generation(data["object_classifier_directory"]["test_img_dir"], (55,27))
    np.save("train_array.npy",train_data)
    np.save("test_array.npy",test_data)
    np.save("train_label", train_label)
    np.save("test_label", test_label)
    print(train_data.shape)
    print("Final Image Array for Classification Created")

In [6]:
data_preparation_pipeline()

(55, 27)
(55, 27)
(10536, 27, 55)
Final Image Array for Classification Created
